딥러닝모델 BERT에 사용할\
한국어 워드피스 분철하는거 연습하는 공간

In [36]:
#정제된 텍스트를 메캅으로 먼저 토큰화한다. => 교착어인 한국어의 단점을 보완하는 방법
from konlpy.tag import Mecab
mecab=Mecab(dicpath=r'C:/mecab/mecab-ko-dic')
clean_texts= ['안녕하세요 나는 김찬란입니다 방구 뿡뿡','이 영화 정말 재미있어요']
#여기서 '나는'이 워드피스 전처리후에도 그대로 나오기때문에 분리해주어야하는데 
#그게 메캅을 지금하는 이유이다. '나' 와 '는' 은 각각의 뜻이 있다.
#kiwi라는 토크나이저를 사용하면 더정확한 분해를 얻을수 있다고 한다.

tokenized_clean_texts =[ mecab.morphs(i) for i in clean_texts ]
print(tokenized_clean_texts)

#토큰화된걸 ' ' 공백한칸을 기준으로 다시 합쳐준다.
rejoined_texts = [' '.join(i) for i in tokenized_clean_texts]
print(rejoined_texts)
labels =[0,1]

[['안녕', '하', '세요', '나', '는', '김', '찬란', '입니다', '방구', '뿡뿡'], ['이', '영화', '정말', '재미있', '어요']]
['안녕 하 세요 나 는 김 찬란 입니다 방구 뿡뿡', '이 영화 정말 재미있 어요']


In [37]:
from transformers import BertTokenizer

# BertTokenizer : 버트에 사용될 워드피스 토큰화를 수행한다.
# bert-base-multilingual-cased : 
# base는 12개 인코더(encoder)층이 있는 버전
# multilingual은 여러언어를 사전학습한 모델이라는 뜻이다.
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# # 텍스트 토큰화 함수  # 코느화되면 어떻게 되는지 확인해본것
# def wordpiece_tokenize(text):
#     tokens = tokenizer.tokenize(text)
#     return tokens

# wordpiece_tokenized =[]
# for i in rejoined_texts:
#     wordpiece_tokenized.append(wordpiece_tokenize(i))
# print(wordpiece_tokenized)


In [40]:
import tensorflow as tf
max_length = 64
def preprocess(texts, labels):
    input_ids =[]
    attention_masks = []
    for i in rejoined_texts:
        input_tokens = tokenizer.batch_encode_plus(
            i,
            max_length=max_length,
            padding="max_length",
            truncation=True,
            return_tensors="tf",
            add_special_tokens=True, 
            return_attention_mask=True
        )
        input_ids.append(input_tokens['input_ids'][0])
        attention_masks.append(input_tokens['attention_mask'][0])
    # 차원 변경
    input_ids = tf.convert_to_tensor(input_ids)
    attention_masks = tf.convert_to_tensor(attention_masks)
    labels = tf.convert_to_tensor(labels)
    return {'input_ids':input_ids, 'attention_masks':attention_masks}, labels

preprocess(rejoined_texts,labels)

# print(tf.convert_to_tensor(input_ids))
# print(input_ids)
# # print(attention_masks)

({'input_ids': <tf.Tensor: shape=(2, 64), dtype=int32, numpy=
  array([[ 101, 9521,  102,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0],
         [ 101, 9638,  102,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0]])>,
  'attention_masks': <tf.Tensor: shape=(2, 64

In [ ]:
from sklearn.model_selection import train_test_split
# 데이터 분할
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# 전처리 적용
train_data, train_labels = preprocess(train_texts, train_labels)
test_data, test_labels = preprocess(test_texts, test_labels)


In [ ]:
#버트 분류모델 불러오기
from transformers import TFBertForSequenceClassification

# 모델 생성
model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
# optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

# 모델 학습
model.fit(train_data, train_labels, epochs=10, batch_size=64)

# 모델 평가
results = model.evaluate(test_data, test_labels)
print("Test loss, accuracy:", results)

In [ ]:
model.summary()